In [ ]:
#  Settings:
#     Need a public or private accessible storage bucket on GCP or similar ..
#
#  From,
#     https://stackoverflow.com/questions/36314797/write-a-pandas-dataframe-to-google-cloud-storage-or-bigquery
#     https://stackoverflow.com/questions/29325458/dictionary-column-in-pandas-dataframe/29325954#29325954

import os
from google.cloud import storage
import pandas as pd

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/work/02_MyKeys.json"

l_bucket = "farrell-bucket"
# l_bucket = "farrell-private"

print ("--")

In [ ]:
#  Setup for all work below ..

df = pd.DataFrame(data=[{1,2,3},{4,5,6}],columns=['a','b','c'])

g_client = storage.Client()

g_bucket = g_client.get_bucket(l_bucket)

print("--")

In [ ]:
#  Test above ..

g_bucket.blob('jira_test/test.csv').upload_from_string(df.to_csv(), 'text/csv')

print("--")

In [ ]:
# ##################################################################

In [ ]:
import os

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)

In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")

In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)

In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)

In [ ]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)

In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
# ##################################################################

In [ ]:
#  Simple test, write, simple data types ..

df = pd.DataFrame([
   [101, "Abhi"  , "Katana"                , 'NY'     ],
   [102, "Justin", "Katana"                , 'LA'     ],
   [103, "Brian" , "Katana"                , 'Denver' ],
   ], columns = ['Col A', 'Col B', 'Col C', 'Col D'])

l_file = "jira_test/test.parquet"

g_bucket.blob(l_file).upload_from_string(df.to_parquet(engine="pyarrow", version="2.6"), "application/octet-stream")

print("--")

In [ ]:
#  Simple test, read output from above ..

l_InputFile  = "gs://" + l_bucket + "/" + l_file

l_df = pd.read_parquet(l_InputFile)

l_df.head(5)

In [ ]:
#  We need a header/schema file to import into a graph ..

df_h = pd.DataFrame([
   ["KATANA_DEFAULT_LABEL=Vertice1"                       ],
   ["id:ID(Vertice1)|name:STRING|other:STRING|city:STRING"],
   ["jira_test/test.parquet"                              ],
   ], columns = ["Col_A"])

l_SchemaFile = "jira_test/test_header.txt"

g_bucket.blob(l_SchemaFile).upload_from_string(df_h.to_csv(header=None, index=None, sep=" "), "text/plain")

print("--")

In [ ]:
#  Make a graph from all above ..

l_inp  = "gs://" + l_bucket + "/jira_test/test_header.txt"
   #
l_data = "gs://" + l_bucket 


import_data.parquet(
    my_graph,
    input_node_path  = l_inp,
    input_dir        = l_data,
    schema_delimiter = "|",
    ids_are_integers = False
)

print("--")

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
l_result = my_graph.query("""

   MATCH (n)
   RETURN n
   
   """)

print(l_result)

In [ ]:
# ##################################################################

In [ ]:
#  Reset from above, delete and recreate the graph

#  DELETE GRAPH
#
for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      l_handle=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

print("--")

In [ ]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=2)

print(my_graph)

In [ ]:
# ##################################################################

In [ ]:
#  Redo most things above, but with more complex data ..

In [ ]:

#  Write the more complex dataframe ..

df = pd.DataFrame([
   [101, 30, {"ab":1, "we":2, "as":3}, "String1"],
   [102, 31, {"ab":4, "we":5, "as":6}, "String2"],
   [103, 32, {"ab":7, "we":8, "as":9}, "String2"],
   ], columns = ["Col A", "Col B", "Col C", "Col D"])

l_file = "jira_test/test2.parquet"

g_bucket.blob(l_file).upload_from_string(df.to_parquet(engine="pyarrow", version="2.6"), "application/octet-stream")

print("--")

In [ ]:
l_InputFile  = "gs://" + l_bucket + "/" + l_file

l_df = pd.read_parquet(l_InputFile)

l_df.head(5)

In [ ]:
#  Prove that we can (parse) the dataframe above, versus say "Col C" being a string ..

l_df2 = pd.DataFrame(columns=['AB','WE','AS'])

for (i,r) in l_df.iterrows():
    e = r['Col C']
    l_df2.loc[i] = [e['ab'], e['we'], e['as']]

print(l_df2)

In [ ]:
df_h = pd.DataFrame([
   ["KATANA_DEFAULT_LABEL=Vertice1"  ],
   ["id:ID(Vertice1)|col2|other|city"],
   ["jira_test/test2.parquet"        ],
   ], columns = ["Col_A"])

l_SchemaFile = "jira_test/test_header3.txt"

g_bucket.blob(l_SchemaFile).upload_from_string(df_h.to_csv(header=None, index=None, sep=" "), "text/plain")

print("--")

In [ ]:
#  Make a graph from all above ..

l_inp  = "gs://" + l_bucket + "/jira_test/test_header3.txt"
   #
l_data = "gs://" + l_bucket


import_data.parquet(
    my_graph,
    input_node_path=l_inp,
    input_dir=l_data,
    schema_delimiter="|",
    ids_are_integers=True
)

print("--")

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:

l_result = my_graph.query("""

   MATCH (n)
   RETURN n
   """)

print(l_result)


In [ ]:
# ##################################################################

In [ ]:
#  Basically above, instead of dictionary, use an array ..
#
#  Go above: Delete graph, recreate graph, then ..

#  Write the more complex dataframe ..

df = pd.DataFrame([
   [101, 30, [ 1, 2, 3 ], "String1"],
   [102, 31, [ 4, 5, 6 ], "String2"],
   [103, 32, [ 7, 8, 9 ], "String2"],
   ], columns = ["Col A", "Col B", "Col C", "Col D"])

l_file = "jira_test/test4.parquet"

g_bucket.blob(l_file).upload_from_string(df.to_parquet(engine="pyarrow", version="2.6"), "application/octet-stream")

print("--")


In [ ]:
l_InputFile  = "gs://" + l_bucket + "/" + l_file

l_df = pd.read_parquet(l_InputFile)

l_df.head(5)

In [ ]:
df_h = pd.DataFrame([
   ["KATANA_DEFAULT_LABEL=Vertice1"  ],
   ["id:ID(Vertice1)|col2|other|city"],
   ["jira_test/test2.parquet"        ],
   ], columns = ["Col_A"])

l_SchemaFile = "jira_test/test_header5.txt"

g_bucket.blob(l_SchemaFile).upload_from_string(df_h.to_csv(header=None, index=None, sep=" "), "text/plain")

print("--")

In [ ]:
#  Make a graph from all above ..

l_inp  = "gs://" + l_bucket + "/jira_test/test_header5.txt"
   #
l_data = "gs://" + l_bucket


import_data.parquet(
    my_graph,
    input_node_path=l_inp,
    input_dir=l_data,
    schema_delimiter="|",
    ids_are_integers=True
)

print("--")

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
l_result = my_graph.query("""

   MATCH (n)
   RETURN n
   
   """)

print(l_result)

In [ ]:
# ##################################################################

In [ ]:

#  Below here, just DF manipulation
#

# Create dataframe where you'll store the dictionary values
# ddf = pd.DataFrame(columns=['AB','WE','AS'])
# 
# Populate ddf dataframe
# for (i,r) in df.iterrows():
#     e = r['Col C']
#     ddf.loc[i] = [e['ab'], e['we'], e['as']]
# 
# Replace df with the output of concat(df, ddf)
# df = pd.concat([df, ddf], axis=1)

# New column order, also drops old Col C column
# df = df[['Col A', 'Col B', 'AB', 'WE', 'AS', 'Col D']]

